<a href="https://colab.research.google.com/github/dltmdgh579/FactCheck_Service/blob/master/%EB%84%A4%EC%9D%B4%EB%B2%84_%EB%89%B4%EC%8A%A4_%EB%8C%93%EA%B8%80_%EC%8A%A4%ED%81%AC%EB%9E%98%ED%95%91.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !pip install konlpy
# !pip install boto3

In [ ]:
import boto3
import pandas as pd
import numpy as np
from konlpy.tag import Okt
import sys
import requests
from bs4 import BeautifulSoup
import time
import random

if sys.version_info[0] < 3:
    from io import StringIO # Python 2.x
else:
    from io import StringIO # Python 3.x

# get your credentials from environment variables
aws_id = ''
aws_secret = ''

client = boto3.client('s3', aws_access_key_id=aws_id,
        aws_secret_access_key=aws_secret)

bucket_name = 'snucsv'

object_key = 'SNU_Data.csv'
csv_obj = client.get_object(Bucket=bucket_name, Key=object_key)
body = csv_obj['Body']
csv_string = body.read().decode('cp949')

df = pd.read_csv(StringIO(csv_string))


In [ ]:
df

,document,label
0,가상화폐 투자자 보호법은 없다,1
1,미국 성인 절반이 코로나 백신을 최소 1차례 맞았다,1
2,세계에 부는 K-신문 '열풍'은 부수 부풀리기의 민낯이다,1
3,"일본 역사 교과서, '독도는 일본 땅'이라는 내용 있어야 검정 통과한다",1
4,"외국인 투표, 한국과 달리 중국과 일본에서는 불가능하다",1
...,...,...
425,"""10억 원으로는 서울 마포구에서 집을 살 수 없었다.""",0
426,"""21대 국회는 1987년 민주화 이후 처음으로 국회법을 준수해 개원하게 된다""",0
427,"""15대 국회의원 출신은 이제 저 밖에 남지 않았습니다.""",0
428,"""이용수 할머니 배후에 곽상도 의원이 있다""",0


In [ ]:
# !pip install boto3
# !pip install konlpy

In [ ]:
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']

In [ ]:
df['document'] = df['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
# 한글과 공백을 제외하고 모두 제거

In [ ]:
okt = Okt()

In [ ]:
df_q = []
for sentence in df['document']:
    temp_X = okt.morphs(sentence, stem=True) # 토큰화
    temp_X = [word for word in temp_X if not word in stopwords] # 불용어 제거
    df_q.append(temp_X)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from konlpy.tag import Okt
import pandas as pd
import os
import time

okt = Okt()
vectorizer = TfidfVectorizer()
df_qq= []

# =========================================================================
# Vectorizer의 argument인 tokenizer에 KoNLPy의 pos 함수로 대체.
class MyTokenizer:
    def __init__(self, tagger):
        self.tagger = tagger

    def __call__(self, sent):
        pos = self.tagger.pos(sent)
        clean_words = []  # 정제된 단어 리스트
        for word in pos:
            # word[1]은 품사를 의미하며, 여기서는 조사, 문장기호, 접두사, Foreign('\n'을 빼주기 위함)인 것은 제외시킴.
            if word[1] not in ['Josa', 'Punctuation', 'Suffix', 'Foreign']:
                if len(word[0]) >= 2:  # 한 글자인 단어들도 의미가 없는 경우가 많으므로 일단 제외.
                    clean_words.append(word[0])
        return clean_words


# =========================================================================

# sentences = ['《알  림》','핸드폰 “010 에 대하여”','나도 몰랐고 모두 다 몰랐던 사실입니다.!','  “정부와 통신사의 합작품“', '핸드폰 전화번호부에 010 번호를 삭제하고', '핸드폰을 걸면 통화료가 30% 싸집니다..', '핸드폰 전화번호부에 010 국번을 지워서', '모두 저장 해두세요.', '또한,', '전해?야 겠네요.', '핸드폰 전화번호 입력하고 핸드폰 전화번호를 걸때에도 010을 빼고 걸고 통화를 해야 핸드폰 전화요금이 30%나 저렴하고 싸게 나옵니다.', '그리고 010을 빼고 전화를 걸면 전화도 더 빨리 연결됩니다.', '정부가 핸드폰 국번 번호를 010으로 통합하면서', '효율적인 통화가 되도록 해놓고,~~', '핸드폰 통신사 눈치를 보느라고 홍보를 지금까지 안하고 있습니다.', '직접실험 전화를 해 보세요. 통화 연결도 빨리 연결되고, 정말 전화가 훨씬 잘 걸립니다.', '생각해보세요 공통된 010 뭐하러 쓰겠어요.', '*지인들에게 많이 알려주세요,*']

my_tokenizer = MyTokenizer(Okt())
tfidf_Vectorizer = TfidfVectorizer(tokenizer=my_tokenizer, min_df=1)    # df 값(단어가 몇 문장들에서 등장하였는지)을 최소 'min_df' 값으로 설정.
for a in range(0, 5):
    X = tfidf_Vectorizer.fit_transform(df_q[a]).toarray()
    # print(X.shape)      # X(2차원 배열)의 행,열 수를 출력.
    # print(tfidf_Vectorizer.vocabulary_)     # 각 단어들이 배열 X에서 몇번째 열(인덱스 값)에 해당하는지 출력.

    # ==========================================================================

    # pandas를 활용하여 각 단어들의 각 문장에서의 tf-idf 값들을 모두 더하고, 내림차순으로 정렬하여 상위 n개 출력.
    count = X.sum(axis=0)           # 2차원 배열 X에서 각 열을 기준으로 합을 구함. (각 단어들의 '최종' tf-idf 값으로 간주.)
    word_count = pd.DataFrame({
        '단어' : tfidf_Vectorizer.get_feature_names(),
        '빈도' : count.flat})
    sorted_df = word_count.sort_values('빈도', ascending=False)

    # print(sorted_df.head(10), "\n")

    # ==========================================================================
    word__ = ""
    word_ = list(np.array(sorted_df['단어'].tolist()))
    for i in range(1, min(len(word_), 7)):
        word__ += word_[i]
        word__ += " "

    
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.90 Safari/537.36'}
    # filename = "SNU_fake_final.csv"
    # f = open(filename, "w", encoding="utf-8-sig", newline="")
    # writer = csv.writer(f)

    # title_ = ["document", "link"]
    # writer.writerow(title_)

    allComments = []
    def create_soup(url):
        res = requests.get(url, headers=headers)
        res.raise_for_status()
        soup = BeautifulSoup(res.text, 'html.parser')
        return soup

    List = []
    # 네이버 뉴스 url을 입력합니다.
    url1 = "https://search.naver.com/search.naver?where=news&sm=tab_jum&query={}".format(word__)
    soup = create_soup(url1)
    for i in range(1, 20):
        try:
            url = soup.select_one("#sp_nws{} > div.news_wrap.api_ani_send > div > div.news_info > div.info_group > a:nth-of-type(2)".format(i))['href']
            oid = url.split("oid=")[1].split("&")[0] #422
            aid = url.split("aid=")[1] #0000430957
            page = 1
            header = {
                "User-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/65.0.3325.181 Safari/537.36",
                "referer": url,
            }
            # time.sleep(random.randint(1,2))
            
            while True:
                c_url = "https://apis.naver.com/commentBox/cbox/web_neo_list_jsonp.json?ticket=news&templateId=default_society&pool=cbox5&_callback=jQuery1707138182064460843_1523512042464&lang=ko&country=&objectId=news" + oid + "%2C" + aid + "&categoryId=&pageSize=20&indexSize=10&groupId=&listType=OBJECT&pageType=more&page=" + str(
                    page) + "&refresh=false&sort=FAVORITE"
                # 파싱하는 단계입니다.
                r = requests.get(c_url, headers=header)
                cont = BeautifulSoup(r.content, "html.parser")
                total_comm = str(cont).split('comment":')[1].split(",")[0]
            
                match = re.findall('"contents":([^\*]*),"userIdNo"', str(cont))
                # 댓글을 리스트에 중첩합니다.
                List.append(match)
                # 한번에 댓글이 20개씩 보이기 때문에 한 페이지씩 몽땅 댓글을 긁어 옵니다.
                if int(total_comm) <= ((page) * 5):
                    break
                else:
                    break
            
            
            # 여러 리스트들을 하나로 묶어 주는 함수입니다.
            def flatten(l):
                flatList = []
                for elem in l:
                    # if an element of a list is a list
                    # iterate over this list and add elements to flatList
                    if type(elem) == list:
                        for e in elem:
                            flatList.append(e)
                    else:
                        flatList.append(elem)
                return flatList
            
            
            # 리스트 결과입니다.


            allComments = flatten(List)
        except:
            pass

    df_qq.append(allComments)
    # word_SNU1 = word_[1]
    # word_SNU2 = word_[2]

df_qq[:5]

[['"근디 세금은 왜 걷어가나요"',
  '"보호도 안해주는데 왜 세금 때가요?? 앞뒤가 안맞네 ㅋㅋㅋ"',
  '"노름하다 돈 잃었다고 정부에 보전해달라는 말을 하는 바보가 어디있겠냐 ㅋㅋ"',
  '"당신은 그 아가리 좀 조심하시길"',
  '"은성수야 공매도 어떻게 할거니 모가지 내놔라 삼성전자가 비틀거린다 감당 되냐??"',
  '"병시나 가상화폐 투자자들이 원하는건 돈 잃고 그돈 복구시켜달라는게 아니잖아 주식시장도 상폐 되거나 돈먹고 튀면 돈 복구 시켜주냐? 세금 낼테니까 비트바이 코리아 같은 다단계 거래소 단속해서 없애고 메이저 거래소들 가끔 입출금 막고 서버 트래픽 핑계로 매수 매도 조작하고있는게 그거 감시해서 추징금 때리고 이소리지"',
  '"주식은 보호해주냐? 공매도량 봤냐? 어디서 혓바닥 놀리노. 혓바닥 짤라뿔라"',
  '"주식 공매도나 어떻게해봐 인간아"',
  '"코인 손해를 보호 해달라는게 아니자나\\n코인법 기준을 만들어서 \\n더러운 애들 처리 해달라는 거자나\\n왜 이해를 못하지?\\n수익이 있으면 세금내는건 맞는데\\n세금을 걷으면 법을 만들어주던가\\n틀을 만들어서 양성화를 시켜줘야지\\n법 만들어서 세력들 좀 조지고 그러면\\n코인판도 거품 가라앉겠지\\n난 뭐 그렇게 생각한다"',
  '"가격 변동으로 인한 이익을 세금으로는 왜 가져가나?? 내로남불 쩐다 쩔어"',
  '"보호 안 해준다는데 세금은 왜 걷어가?? 갈취잖아"',
  '"이런넘이 한나라의 금융수장이라니..ㅉㅉ"',
  '"주식은 망하면 언제 국가에서 구제해줬소?? 그냥 관심 끄시오"',
  '"누가 가격변동 보호 해달랬나 ㅋㅋㅋㅋㅋㅋ 주식은 가격변동에 완전 보호냐?ㅋㅋㅋㅋ개풀뜯어먹누"',
  '"다들 궁금한게  하나같이 세금은 왜 걷나? 명확한 답변 기대함"',
  '"보호안해줘도  암말안하니까 그입다물라..."',
  '"자리세는 뜯어가면서 보호는 못해주겠다?  날강도네"',
  '"코인 떡상하고 떡락하는걸 국가가 왜보호해줘,,, 주식 떡락하는거 책임져주

In [ ]:
import pandas as pd
from konlpy.tag import Okt
import numpy as np
import boto3
import sys
from gensim.models import FastText

docs = list(np.array(df['document'].tolist()))
print(docs)
okt = Okt()


# =========================================================================
# Vectorizer의 argument인 tokenizer에 KoNLPy의 pos 함수로 대체.
class MyTokenizer:
    def __init__(self, tagger):
        self.tagger = tagger

    def __call__(self, sent):
        pos = self.tagger.pos(sent)
        clean_words = []  # 정제된 단어 리스트
        for word in pos:
            # word[1]은 품사를 의미하며, 여기서는 조사, 문장기호, 접두사, Foreign('\n'을 빼주기 위함)인 것은 제외시킴.
            if word[1] not in ['Josa', 'Punctuation', 'Suffix', 'Foreign']:
                if len(word[0]) >= 2:  # 한 글자인 단어들도 의미가 없는 경우가 많으므로 일단 제외.
                    #if word[0] not in ['있다', '했다', '한다', '없다', '된다']:
                    clean_words.append(word[0])
        return clean_words

# =========================================================================

# 사전 학습된 word2vec 모델 호출.
ft = FastText.load_fasttext_format('D:/pythonProject/FakeNewsFiltering/wiki.ko.bin', encoding='utf-8')
result = ft.wv.most_similar('문재인')
k = 0
# for i in range(3):
#     k = k + result[i][1]
# result_average = k/3
# print(result)

my_Tokenizer = MyTokenizer(Okt())
words_vector_list = []
for a in range(len(docs)):
    token_words = my_Tokenizer(docs[a])
    line = []
    for b in range(len(token_words)):
        print(b)
        print(token_words[b])
        similar_top10_vector = ft.wv.most_similar(token_words[b])
        words_vector_value = (similar_top10_vector[0][1] + similar_top10_vector[1][1] + similar_top10_vector[2][1])/3
        line.append(words_vector_value)
    words_vector_list.append(line)
    print(words_vector_list)

# =========================================================================